# EDA & Model Development 

In [165]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sb
import random

## Initial Setup

In [166]:
#https://raw.githubusercontent.com/jchristo12/fantasy_football/master/data/full_data.csv
df = pd.read_csv('https://github.com/jchristo12/fantasy_football/blob/master/data/full_data.csv?raw=true')

In [167]:
df.columns

Index(['pk', 'gid', 'seas', 'wk', 'player', 'fname', 'lname', 'full_name',
       'team', 'pos1',
       ...
       'ou', 'sprv', 'ptsv', 'ptsh', 'udog', 'gen_cond', 'udog_binary',
       'gen_dv', 'def_team', 'f_pts'],
      dtype='object', length=171)

In [168]:
#remove rows that have NaN for the shifted variables
df_clean = df[~df.loc[:,'seas_pa':'seas_tdret'].isna().all(axis=1)]

##### We are only focused on offensive players right now. Therefore, we will specifiy the positions we want to retain in the data and remove everyone else

In [169]:
#store positions we are concerned about; will use these to filter out 
pos_of_interest = ['QB', 'RB', 'WR', 'TE']
#filter out positions we don't care about
df_clean2 = df_clean[df_clean['pos1'].isin(pos_of_interest)]

##### Convert each column to the appropriate data type (i.e. make the categorical data categories)

In [170]:
#set the column types
col_dtypes = {'category': ['seas', 'wk', 'pos1', 'team', 'udog', 'v', 'h', 'day', 'stad', 'wdir',
                          'surf', 'gen_cond', 'gen_dv', 'def_team']}
#flip the key and values around so they will work in the argument for 'astype()'
col_dtypes_alt = {old: new for new, old_all in col_dtypes.items() for old in old_all}

In [171]:
df_clean2 = df_clean2.astype(col_dtypes_alt)

##### Remove rookies

In [172]:
#store dataframe of non-rookies
df_clean2_vet = df_clean2.loc[df_clean2['exp']!=1, :]
#store dataframe of rookie data
df_clean2_rook = df_clean2.loc[df_clean2['exp']==1, :]

##### Segment out the WR data

Non-rookies data only

In [173]:
df_wr = df_clean2_vet.loc[df_clean2_vet['pos1']=='WR', :]

##### Only focus on week 10 data

In [174]:
df_wr10 = df_wr.loc[df_wr['wk'] == 10, :]

In [175]:
df_wr10.shape

(1860, 171)

## EDA

Only focusing on the WR data right now. Will need to build this so that each action is extendable to the other positions

### Training/Test split

In [176]:
from sklearn.model_selection import train_test_split

In [177]:
#set the random seed for reproducability
random.seed(837)

In [215]:
#break out the data between training and test
train_wr, test_wr = train_test_split(df_wr10, train_size=0.75, test_size=0.25, shuffle=True)

In [179]:
#shape of the data
print(train_wr.shape, test_wr.shape)

(1395, 171) (465, 171)


In [216]:
#reset index on both dataframes
train_wr = train_wr.reset_index(drop=True)
test_wr = test_wr.reset_index(drop=True)

### Missing Data analysis

#### Basic analysis

In [217]:
#create a series of percent of missing data
missing_data_pct = train_wr.isna().sum() / train_wr.shape[0]

In [218]:
#list all columns with missing data that is greater than 25%
missing_data_pct[missing_data_pct > 0.25].sort_values(ascending=False)

bench       0.718996
hand        0.629391
arm         0.612186
cone        0.523297
shuttle     0.509677
broad       0.482437
vertical    0.440860
forty       0.376344
dtype: float64

In [219]:
#columns with missing values but less than or equal to 25%
impute_cols = missing_data_pct[(missing_data_pct <= 0.25) & (missing_data_pct > 0)].sort_values(ascending=False)
impute_cols

temp    0.153405
humd    0.033692
wdir    0.025090
dtype: float64

#### Take action on missing data

##### Drop columns with too many missing values

In [220]:
#drop columns with missing data greater than 25%
#store the column names
missing_cols_del = missing_data_pct[missing_data_pct > 0.25].sort_values(ascending=False).index
#drop the columns and store as new dataframe
train_wr_miss = train_wr.drop(missing_cols_del, axis=1, inplace=False)

In [221]:
#check the shape
train_wr_miss.shape

(1395, 163)

##### Impute the rest of the missing values

In [222]:
from sklearn.impute import SimpleImputer

In [223]:
#Build simple imputers for both numeric and categorical features
numeric_impute = SimpleImputer(missing_values=np.NaN, strategy='median')
cat_impute = SimpleImputer(missing_values=np.NaN, strategy='most_frequent')

In [224]:
#create a dataframe of all of the features to impute
missing_values_df = train_wr_miss.drop(train_wr_miss.columns.difference(impute_cols.index), axis=1, inplace=False)

In [225]:
missing_values_df.columns

Index(['temp', 'wdir', 'humd'], dtype='object')

In [226]:
#store the columns that need to use the numeric imputation and the categorical imputation
impute_numeric_col = missing_values_df.select_dtypes(include=np.number).columns
impute_cat_col = missing_values_df.select_dtypes(exclude=np.number).columns

This code below attempts to handle infinity values; This is not an issue right now but something to be aware of

In [227]:
#Boolean if the column has 'inf' values or not
#inf_cols = np.isinf(train_wr_miss.loc[:, impute_numeric_col]).any()

In [228]:
#list of columns that have 'inf' values
#inf_cols2 = list(train_wr_miss.loc[:, impute_numeric_col].columns.to_series()[inf_cols].values)

Continue with simple imputation

In [229]:
#impute numeric columns
imputed_numeric_df = pd.DataFrame(numeric_impute.fit_transform(train_wr_miss.loc[:, impute_numeric_col]), columns=impute_numeric_col).add_prefix('imp_')

In [230]:
#impute categorical columns
imputed_cat_df = pd.DataFrame(cat_impute.fit_transform(train_wr_miss.loc[:, impute_cat_col]), columns=impute_cat_col).add_prefix('imp_')

In [234]:
#add imputed columns to original data frame
train_wr_imputed = pd.concat([train_wr_miss, imputed_numeric_df, imputed_cat_df], axis=1)